In [ ]:
import pickle
import numpy as np
import pandas as pd
import gc


with open('D:/inputs/24/train.pickle', 'rb') as f:
    train = pickle.load(f)
with open('D:/inputs/24/train_y.pickle', 'rb') as f:
    train_y = pickle.load(f)
train_y = train_y.reshape(75957, 40, 40, 1)    
with open('D:/inputs/24/test.pickle', 'rb') as f:
    test = pickle.load(f)


In [ ]:
from sklearn.metrics import f1_score
def mae_over_fscore(y_true, y_pred):
    '''
    y_true: sample_submission.csv 형태의 실제 값
    y_pred: sample_submission.csv 형태의 예측 값
    '''


    y_true = np.array(y_true)
    y_true = y_true.reshape(1, -1)[0]  
    
    y_pred = np.array(y_pred)
    y_pred = y_pred.reshape(1, -1)[0]
    
    # 실제값이 0.1 이상인 픽셀의 위치 확인
    IsGreaterThanEqualTo_PointOne = y_true >= 0.1
    
    # 실제 값에 결측값이 없는 픽셀의 위치 확인 
    IsNotMissing = y_true >= 0
    
    # mae 계산
    mae = np.mean(np.abs(y_true[IsGreaterThanEqualTo_PointOne] - y_pred[IsGreaterThanEqualTo_PointOne]))
    
    # f1_score 계산 위해, 실제값에 결측값이 없는 픽셀에 대해 1과 0으로 값 변환
    y_true = np.where(y_true[IsNotMissing] >= 0.1, 1, 0)
    
    y_pred = np.where(y_pred[IsNotMissing] >= 0.1, 1, 0)
    
    # f1_score 계산    
    f_score = f1_score(y_true, y_pred) 
    
    # f1_score가 0일 나올 경우를 대비하여 소량의 값 (1e-07) 추가 
    return mae / (f_score + 1e-07) 

In [ ]:
train.shape

In [ ]:
train = train.reshape(train.shape[0] * train.shape[1] * train.shape[2], train.shape[3])
test = test.reshape(test.shape[0] * test.shape[1] * test.shape[2], test.shape[3])
train_y = train_y.reshape(train_y.shape[0] * train_y.shape[1] * train_y.shape[2], train_y.shape[3])

In [ ]:
train.shape

In [ ]:
train = pd.DataFrame(train)
train.columns = ['temp0', 'temp1', 'temp2', 'temp3', 'temp4', 'temp5', 'temp6', 'temp7', 'temp8', 'surface', 'GMI_y', 'GMI_x', 'DPR_y', 'DPR_x']
train['surface'] = train['surface'].astype('int16')

test = pd.DataFrame(test)
test.columns = ['temp0', 'temp1', 'temp2', 'temp3', 'temp4', 'temp5', 'temp6', 'temp7', 'temp8', 'surface', 'GMI_y', 'GMI_x', 'DPR_y', 'DPR_x']
test['surface'] = train['surface'].astype('int16')

train_y = pd.DataFrame(train_y).rename(columns={0:'precipitation'})

In [ ]:
train.head()

In [ ]:
del train['temp1'], train['temp3']
# train.corr()

In [ ]:
train['GMI_x_plus_y'] = train['GMI_x'] + train['GMI_y']
train['GMI_x_minus_y'] = train['GMI_x'] - train['GMI_y']
train['DPR_x_plus_y'] = train['DPR_x'] + train['DPR_y']
train['DPR_x_minus_y'] = train['DPR_x'] - train['DPR_y']

In [ ]:
train.head()

In [ ]:
train = pd.concat([train, train_y], axis = 1)

In [ ]:
train.head()

In [ ]:
train.groupby('surface')['precipitation'].mean()

In [ ]:
train['surface'].value_counts().sort_index()

In [ ]:
pd.options.display.max_rows = 100
pd.options.display.max_columns = 100

In [ ]:
train['surface_group'] = train['surface'].copy()

In [ ]:
train['surface_group'] = (train['surface_group'] / 100).astype('int16')

In [ ]:
# train.groupby('surface_group')['precipitation'].mean()

In [ ]:
train[train['precipitation'] > 0].groupby('surface_group')['precipitation'].mean()

In [ ]:
# 비 내린 픽셀 비율
# len(train[train['precipitation'] > 0]) / len(train)
113352550 / 121553600

In [ ]:
train.head()

In [ ]:
target = train['precipitation'].copy()
del train['precipitation']

In [ ]:
train = train.drop(['surface'], axis=1)

In [ ]:
train.head()

In [ ]:
# from sklearn.preprocessing import StandardScaler
# from sklearn.preprocessing import RobustScaler

# ss = StandardScaler()
# # rs = RobustScaler()
# train = ss.fit_transform(train)

# del ss

In [ ]:
# import pickle

# with open('inputs/train_ss.pickle', 'wb') as f:
#     pickle.dump(train, f, protocol=4)
    
# with open('inputs/train_ss.pickle', 'rb') as f:
#     train = pickle.load(f)
    
# with open('D:/inputs/24/train_y.pickle', 'rb') as f:
#     train_y = pickle.load(f)
# train_y = train_y.reshape(75957, 40, 40, 1)    
# train_y = train_y.reshape(train_y.shape[0] * train_y.shape[1] * train_y.shape[2], train_y.shape[3])

### RandomForest Regressor 모델

In [ ]:
# from sklearn.ensemble import RandomForestRegressor
# rf = RandomForestRegressor(criterion='mae', verbose=1, n_jobs=-1, max_depth=5)
# rf.fit(train, target)
# print(mae_over_fscore(target, rf.predict(train)))

### Lidge 모델

In [ ]:
# from sklearn.linear_model import RidgeCV, Ridge, Lasso
# from sklearn.metrics import mean_absolute_error

# for alpha in [1, 10, 50, 100, 200, 300]:
#     ridge = Ridge(alpha=alpha)
#     ridge.fit(train, target)
#     print(alpha)
#     print(mae_over_fscore(target, ridge.predict(train)))
#     print('\n')

### Lasso 모델 

In [ ]:
# for alpha in [1, 10, 100]:
#     lasso = Lasso()
#     lasso.fit(train, target)
#     print(mae_over_fscore(target, lasso.predict(train)))

### Catboost Regressor 모델

In [ ]:
from catboost import CatBoostRegressor
# category 데이터에 특히 잘 적용되고,
# 컬럼 랜덤하게 뽑고, row도 랜덤하게 뽑아서 랜덤포레스트와 비슷한 느낌. 파라미터 설정에 신경쓰지 않아도 어느 정도점수 잘나오는 모델

cb = CatBoostRegressor(iterations=10, loss_function='MAE')
cb.fit(train, target)
print(mae_over_fscore(target, cb.predict(train)))

In [ ]:
cb.fit(train, target, init_model=cb)
print(mae_over_fscore(target, cb.predict(train)))

In [ ]:
cb.fit(train, target, init_model=cb)
print(mae_over_fscore(target, cb.predict(train)))

In [ ]:
cb.save_model("cb_model_40",
           format="cbm",
           export_parameters=None,
           pool=None)

In [ ]:
# catboost model load - continuous fit 가능
# cb = CatBoostRegressor(iterations=10, loss_function='MAE')
# cb.load_model("cb_model_40")
# cb.fit(train, target, init_model=cb1)

In [ ]:
print(mae_over_fscore(target, cb1.predict(train)))

10 - 3.8450220123151535  
20 - 3.169688157233239  
30 - 2.930901411484702  
40 - 2.7957138063731715

### lightgbm 모델

In [ ]:
from lightgbm import LGBMRegressor

lgb = LGBMRegressor(device='cpu', n_estimators = 40)
lgb.fit(train, target)

In [ ]:
print(mae_over_fscore(target, lgb.predict(train)))

- 10 - 5.615421879934444
- 20 - 2.5261612717784985
- 30 - 2.490089590398271
- 40 - 2.478701832641337

아이디어
- temp0~8를 pca로 축소하기